# Before proceeding into the Agent building part, Let's focus on Langchain

In [1]:
# imports
from google import genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from load_env import gemini_api_key

### Instantiating a Model Object

In [2]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = gemini_api_key,
    temperature = 0.3,
    max_tokens = 1500,
    timeout = None,
    max_retries = 2,
    # some other parameters
)


### Invocation

In [7]:
from pprint import pprint
messages = [
    ("system", "You are a helpful assistant that translates English to Chinese. Translate the user sentence"),
    ("user", "I love programming")
]

response = llm.invoke(messages)
pprint(response)

AIMessage(content='我喜欢编程 (Wǒ xǐhuan biānchéng)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e9f64fac-2d91-4bce-b27e-793b6a4bde28-0', usage_metadata={'input_tokens': 18, 'output_tokens': 14, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}})


In [8]:
pprint(response.content)

'我喜欢编程 (Wǒ xǐhuan biānchéng)'


### Chaining

In [ ]:
# we use ChatPromptTemplate to create a prompt for the model

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """You are a helpful assistant that translates {input_language} to {output_language}.""",
        ),
        ('human', '{input}'),
    ]
)

chain = prompt | llm # what is this doing?

chain.invoke({
    'input_language': 'English',
    'output_language': 'German',
    'input': 'I love programming'
})

AIMessage(content='Ich liebe Programmieren.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e5e8267d-c415-4560-85e7-195d29fdde82-0', usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}})

## Multimodal Usage

Gemini models can accept multimodal inputs (text, images, audio, and video) and 
, for some models, generate multimodal outputs. We will concern ourselves with that in a different notebook

## Tool Calling

You can equip the model with tools to use in its responses.

In [14]:
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

# Define the tool
@tool(description="Get the current weather in a given location")
def get_weather(location: str) -> str:
    """Get the current weather in a given location"""
    return f"The weather in {location} is sunny"

# Initialize the model and bind the tool
llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash', api_key = gemini_api_key)
llm_with_tools = llm.bind_tools([get_weather])

# Invoke the model with a query that should trigger the tool

query = 'What is the weather in Tokyo?'
ai_response = llm_with_tools.invoke(query)

# check the tool calls in the response
pprint(ai_response.tool_calls) # list of tools and relevant arguments

# Example tool call message would be needed here if you were actually using the tool
from langchain_core.messages import ToolMessage
tool_message = ToolMessage(
    content = get_weather(*ai_response.tool_calls[0]['args']),
    tool_call_id = ai_response.tool_calls[0]['id']
)

llm_with_tools.invoke([ai_response,tool_message]) # example of passingtool result back

[{'args': {'location': 'Tokyo'},
  'id': 'd0b8bbaf-be4e-4ae5-a99f-56559ed6fed2',
  'name': 'get_weather',
  'type': 'tool_call'}]


/var/folders/zl/fqyfw3z95bjccqn0m2sm7qhw0000gn/T/ipykernel_42240/2081460996.py:25: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  content = get_weather(*ai_response.tool_calls[0]['args']),


AIMessage(content='OK. The weather in Tokyo is sunny.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d46d7fdc-fcf7-4c77-88ae-9e4a4130ba7a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 10, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}})

## Structured Output

Force the model to respond with a specific structure using Pydantic Models

In [15]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

# Define the desired structure
class Person(BaseModel):
    """Information about a person"""

    name: str = Field(...,description="The name of the person")
    height_m: float = Field(...,description="The height of the person in meters")

# Initialize the model
llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash', api_key = gemini_api_key, temperature=0)
structured_llm = llm.with_structured_output(Person) # this is where you enforce a response

# Invoke the model
query = "Who was the 16th president of the USA and how tall was he?"
response = structured_llm.invoke(query)
print(response)

name='Abraham Lincoln' height_m=1.93


In [30]:
# Invoke the model again
from IPython.display import Markdown, display

query = "What is the weather in Tokyo?"
response = structured_llm.invoke(query)
response # so I am guessing that we get a default response if the model does not know the answer

Person(name='John Doe', height_m=1.8)

In [33]:
# invoke the model again
query = 'Who is the fourth Hokage of the leaf village and how tall was he?'
response = structured_llm.invoke(query)
response

Person(name='Minato Namikaze', height_m=1.75)

## Token Usage Tracking

Access token usage information from the response metadata

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash', api_key = gemini_api_key)

query = 'Explain the concept of prompt engineering in one sentence.'
result = llm.invoke(query)

print(result.content)
print('\nUsage Metadata:')
print(result.usage_metadata)

Prompt engineering is the art and science of crafting effective text inputs to guide large language models toward generating desired outputs.

Usage Metadata:
{'input_tokens': 10, 'output_tokens': 23, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}


## Built-in Tools

Google Gemini supports a varieyt of built-in tools (googlesearch, code execution), which can be bound to the model in the usual way

In [36]:
from google.ai.generativelanguage_v1beta import Tool as GenAI_Tool

resp = llm.invoke(
    'When is the next total solar eclipse in the US and when was the last one?',
    tools = [GenAI_Tool(google_search={})],
)

pprint(resp.content)

('The most recent total solar eclipse in the US was on April 8, 2024. The next '
 'total solar eclipse in the US will occur on March 30, 2033, but it will only '
 'be visible in Alaska. The next total solar eclipse visible in the contiguous '
 'United States will be on August 22, 2044. The path of totality will only be '
 'visible in Montana, North Dakota, and South Dakota. Another total solar '
 'eclipse will occur on August 12, 2045, and will be visible from California '
 'to Florida.')


In [37]:
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool

resp = llm.invoke(
    "What is 2*2, use python",
    tools=[GenAITool(code_execution={})],
)

resp

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_google_genai/chat_models.py:686: UserWarning: 
        ⚠️ Warning: Output may vary each run.  
        - 'executable_code': Always present.  
        - 'execution_result' & 'image_url': May be absent for some queries.  

        Validate before using in production.

  warnings.warn(


AIMessage(content=[{'type': 'executable_code', 'executable_code': 'print(2*2)\n', 'language': <Language.PYTHON: 1>}, {'type': 'code_execution_result', 'code_execution_result': '4\n', 'outcome': <Outcome.OUTCOME_OK: 1>}, '2 * 2 = 4'], additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f4bcef38-1889-4989-9158-f40d8b4e995a-0', usage_metadata={'input_tokens': 9, 'output_tokens': 17, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}})

In [38]:
for c in resp.content:
    if isinstance(c, dict):
        if c["type"] == "code_execution_result":
            print(f"Code execution result: {c['code_execution_result']}")
        elif c["type"] == "executable_code":
            print(f"Executable code: {c['executable_code']}")
    else:
        print(c)

Executable code: print(2*2)

Code execution result: 4

2 * 2 = 4
